# Installations

In [1]:
!git clone https://github.com/cg123/mergekit.git && cd mergekit && git reset --hard c891a0900969c1fb8ce678e3ace0e084a92c24fc && cd ..

Cloning into 'mergekit'...
remote: Enumerating objects: 2528, done.
remote: Counting objects: 100% (981/981), done.
remote: Compressing objects: 100% (269/269), done.
remote: Total 2528 (delta 850), reused 721 (delta 712), pack-reused 1547 (from 3)
Receiving objects: 100% (2528/2528), 853.60 KiB | 15.81 MiB/s, done.
Resolving deltas: 100% (1721/1721), done.
HEAD is now at c891a09 Make Cohere lm_head optional (#417)


In [2]:
!pip install --upgrade pip
%cd mergekit
!pip install -e . -U --quiet --root-user-action=ignore
%cd ..

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 27.6 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
/kaggle/working/mergekit
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 53.1 MB/s eta 0:00:00
  Building editable for mergekit (pyproject.toml) ... done
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain 0.3.12 requires pydantic<3.0.0,>=2.7.4, but you have pydantic 2.7.1 which is incompatible.
/kaggle/working


In [3]:
!git clone -b external_ds_download https://github.com/ahmedamrelhefnawy/lm-evaluation-harness
%cd lm-evaluation-harness
!pip install -q -e . -U
%cd ..

Cloning into 'lm-evaluation-harness'...
remote: Enumerating objects: 29360, done.
remote: Total 29360 (delta 0), reused 0 (delta 0), pack-reused 29360 (from 1)
Receiving objects: 100% (29360/29360), 23.18 MiB | 21.56 MiB/s, done.
Resolving deltas: 100% (19266/19266), done.
/kaggle/working/lm-evaluation-harness
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 80.4 MB/s eta 0:00:00
  Building editable for lm_eval (pyproject.toml) ... done
/kaggle/working


# Login

In [4]:
from huggingface_hub import login
token= "**" # Put your token here
login(token)

# Imports

Enable max download speed from huggingface

In [5]:
import os
os.environ["HF_HUB_ENABLE_HF_TRANSFER"] = "1"

In [6]:
import gc
import os

from transformers import AutoModel, AutoConfig, AutoModelForCausalLM, AutoTokenizer, pipeline, logging
import datasets

import tensorflow as tf
import torch
import numpy as np

import yaml
import json
import copy
import random
import time

from deap import base, tools, creator, algorithms

import matplotlib.pyplot as plt


%cd ./mergekit
from mergekit.config import MergeConfiguration
from mergekit.merge import MergeOptions, run_merge
%cd ..

%cd ./lm-evaluation-harness
from lm_eval import simple_evaluate, downloading_tasks
%cd ..

# Set the verbosity to ERROR to suppress warnings
logging.set_verbosity_error()

/kaggle/working/mergekit
/kaggle/working
/kaggle/working/lm-evaluation-harness
/kaggle/working


# Custom Model Class

In [7]:
class custom_model:
    def __init__(self,
                 base_model: str,
                 dtype: str,
                 num_layers: int,
                 layers: list[int] = None,
                 path: str = "/content/merged") -> None:

        self.base_model = base_model
        self.num_layers = num_layers
        self.layers = layers if layers else [i for i in range(self.num_layers)]
        self.dtype = dtype
        self.path = path

    @property
    def config(self):

        sources = []
        for j in self.layers:
            sources.append(                {
                    'model': self.base_model,
                    'layer_range': [j, j+1]
                })

        config_dict = {
                        'slices': [
                            {
                                'sources': sources
                            }
                        ],
                        'merge_method': 'passthrough',
                        'dtype': self.dtype
                    }
        # Convert dictionary to YAML
        config_text = yaml.dump(config_dict, sort_keys=False, default_flow_style=None) 

        return config_text

    @config.setter
    def config(self, text: str):
        self.config = text

    def remove_layers(self, removed_layers: list[int]):
        for item in removed_layers:
            self.layers.remove(item)
    
    def reset_layers(self):
        self.layers = [i for i in range(self.num_layers)]
        
    def build(self,
              config_yaml_path,
              lora_merge_cache= "/tmp",
              cuda= bool(tf.config.list_physical_devices('GPU')),
              copy_tokenizer=True,
              lazy_unpickle=False,
              low_cpu_memory=False,
              trust_remote_code=True,
              allow_crimes=True):

        # Write Down Config
        with open(config_yaml_path, 'w') as cf:
            cf.write(self.config)

        # Actually do merge
        with open(config_yaml_path, "r", encoding="utf-8") as fp:
            merge_config = MergeConfiguration.model_validate(
                yaml.safe_load(fp))

        run_merge(
            merge_config,
            out_path=self.path,
            options=MergeOptions(

                lora_merge_cache=lora_merge_cache,
                cuda=cuda,
                copy_tokenizer=copy_tokenizer,
                lazy_unpickle=lazy_unpickle,
                low_cpu_memory=low_cpu_memory,
                trust_remote_code=trust_remote_code,
                allow_crimes=allow_crimes
            ),
        )

    def evaluate(self, func, *args):
        return func(*args)

# Genetic Algorithm

## Globals

In [8]:
# Merge Parameters
OUTPUT_PATH = "/content/merged"  # folder to store the result in
LORA_MERGE_CACHE = "/tmp"  # change if you want to keep these for some reason
CONFIG_YAML = "/kaggle/working/config.yaml"  # merge configuration file
COPY_TOKENIZER = True  # you want a tokenizer? yeah, that's what i thought
LAZY_UNPICKLE = False  # experimental low-memory model loader
LOW_CPU_MEMORY = False  # enable if you somehow have more VRAM than RAM+swap

In [9]:
BASE_MODEL_SCORES = {'deepseek-ai/DeepSeek-R1-Distill-Qwen-7B': {'time': 1889.357815027237,
                          'batch_size':2 },
                    }

In [10]:
# Define random seed
random.seed(25)

In [11]:
def clear_cache():
    for var_name in list(globals().keys()):
        var = globals()[var_name]
        if isinstance(var, (AutoModel, AutoTokenizer)):
            del globals()[var_name]

    for var_name in list(locals().keys()):
        var = locals()[var_name]
        if isinstance(var, (AutoModel, AutoTokenizer)):
            del locals()[var_name]
    
    tf.keras.backend.clear_session()
    torch.cuda.empty_cache()
    gc.collect()

## Algorithm

In [12]:
def run_ga(custom_model,
           num_pruned_layers,
           tasks,
           metrics,
           pop_size = 60,
           max_gens = 10,
           tourn_size= 5,
           p_cx= 0.9,
           p_mu= 0.5,
           hall_of_fame = 1,
           prev_results : dict = {}):
    
    custom_model.reset_layers()

    toolbox = base.Toolbox()
    
    generated = set(prev_results)

    def sortedRandomPick(num_layers, num_pruned_layers):        
        chromosome = sorted(random.sample(range(2, num_layers), num_pruned_layers))
    
        while tuple(chromosome) in generated:
            chromosome = sorted(random.sample(range(2, num_layers), num_pruned_layers))
    
        generated.add(tuple(chromosome))
    
        return chromosome

    # Create the LayersPicker operator
    toolbox.register('LayersPicker', sortedRandomPick, custom_model.num_layers, num_pruned_layers)
    
    # Create the fitness class
    creator.create('FitnessMin', base.Fitness, weights=(1,)) # -1 => Minimum || 1 => Maximum
    
    # Create the Chromosome class
    creator.create('Chromosome', list, fitness= creator.FitnessMin)
    
    # Register the ChromosomeCreator operator
    toolbox.register('ChromosomeCreator', tools.initIterate, creator.Chromosome, toolbox.LayersPicker)
    
    # Register the PopulationCreator operator
    toolbox.register('PopulationCreator', tools.initRepeat, list, toolbox.ChromosomeCreator)
    
    
    # Define a function to evaluate the error of the selected model
    evaluated_dict = prev_results
    
    def evaluate(chromosome):    
        # Clear Cache if exists
        try:
            del hfmodel, hfpipe
            clear_cache()
        except:
            pass
    
        # Remove chromosomes with insufficient number of genes
        if len(set(chromosome)) != num_pruned_layers:
            return (0,)
    
        # Chromosome key preprocessing
        chromosome = sorted(chromosome)
        chromosome_key = tuple(chromosome)
    
        # Evaluate the chromosome
        if chromosome_key in evaluated_dict:
            evaluated_dict[chromosome_key]['counter'] += 1
    
            score = evaluated_dict[chromosome_key]['score']
    
        else:
    
            # test_model 
            custom_model.reset_layers()
    
            custom_model.remove_layers(chromosome)
    
            custom_model.build(config_yaml_path= CONFIG_YAML,
                            lora_merge_cache= "/tmp",
                            # cuda= 
                            )

            # Start Timer
            start = time.time()

            # Start task evaluation
            results = simple_evaluate(
                                    model="hf",
                                    model_args= {'pretrained': custom_model.path,
                                                 'trust_remote_code': True},
                                    tasks= ['hellaswag', 'mutual', 'piqa', 'truthfulqa_mc2', 'gsm8k', 'lambada_openai', 'lambada_standard', 'winogrande', 'openbookqa'],
                                    log_samples= True,
                                    device= "cuda:0",
                                    batch_size = "auto:4",
                                    limit = 100,
                                    task_dict = tasks,
                                    apply_chat_template= True,
                                    )

            # Stop Timer and save the result
            model_time = time.time() - start
            base_model_time = BASE_MODEL_SCORES[custom_model.base_model]
            speed_score = (base_model_time**0.5) * (model_time**0.5)
            
            score, n_tasks = 0, 0
            metrics = metrics
            for task, metric in zip(results['results'], metrics):
        
                n_tasks += 1
                task_results = results['results'][task]
                score += task_results[metric] ** 2
            
            score = score / n_tasks
                
            evaluated_dict[chromosome_key] = {}
            evaluated_dict[chromosome_key]['task_score'] = score
            evaluated_dict[chromosome_key]['score'] = score
            
        print("-"*30)
        print(f'Removed: {chromosome} --> Score: {score} %')
        print("-"*30)
        print(" ")
        
        # Save in JSON File
        evaluated_dict_str_keys = {str(k): v for k, v in evaluated_dict.items()}
        with open('/kaggle/working/evaluated_dict.json', 'w') as json_file:
            json.dump(evaluated_dict_str_keys, json_file, indent=4)
        
        return (score,)
    # Register the evaluation function
    toolbox.register('evaluate', evaluate)
    
    # Create the genetic operators
    toolbox.register('select', tools.selTournament, tournsize= tourn_size)
    
    toolbox.register('mate', tools.cxOnePoint)
    
    toolbox.register('mutate', tools.mutUniformInt, low= 0, up= custom_model.num_layers-1, indpb= 1/num_pruned_layers)
    
    # Create the statistics object
    stats = tools.Statistics(lambda chromosome: chromosome.fitness.values)
    
    # Register the statistics object
    stats.register('max_score', np.max)
    stats.register('avg_score', np.mean)
    
    # Create the hall of fame object
    hof = tools.HallOfFame(hall_of_fame)
    
    # To clear Memory if there is residual in it.
    clear_cache()
    
    # Define the algorithm
    population, logbook = algorithms.eaSimple(
        toolbox.PopulationCreator(n= pop_size),
        toolbox,
        cxpb= p_cx,
        mutpb= p_mu,
        ngen= max_gens,
        stats= stats,
        halloffame= hof,
        verbose= True,
    )

In [13]:
ready_tasks = downloading_tasks(tasks= ['hellaswag', 'mutual', 'piqa', 'truthfulqa_mc2', 'gsm8k', 'lambada_openai', 'lambada_standard', 'winogrande', 'openbookqa'])
metrics = ['acc_norm,none', 'mrr,none', 'acc_norm,none', 'acc,none', 'exact_match,strict-match', 'acc,none', 'acc,none', 'acc,none', 'acc_norm,none']

README.md:   0%|          | 0.00/6.84k [00:00<?, ?B/s]

hellaswag.py:   0%|          | 0.00/4.36k [00:00<?, ?B/s]

dataset_infos.json:   0%|          | 0.00/2.53k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/39905 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/10003 [00:00<?, ? examples/s]

...

validation-00000-of-00001.parquet:   0%|          | 0.00/58.2k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/55.5k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/4957 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/500 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/500 [00:00<?, ? examples/s]

In [14]:
# Specify the model name or path
base_model = "deepseek-ai/DeepSeek-R1-Distill-Qwen-7B"

In [15]:
# Load the configuration
config = AutoConfig.from_pretrained(base_model)

dtype = getattr(config, "torch_dtype", None)
dtype = str(dtype)[6:]
print(f"Data type (dtype): {dtype}")

# Get the number of layers
if hasattr(config, "num_hidden_layers"):  # Common for transformer-based models
    num_layers = config.num_hidden_layers
    print(f"Number of layers: {num_layers}")
else:
    print("Number of layers not specified in the config.")

config.json:   0%|          | 0.00/680 [00:00<?, ?B/s]

Data type (dtype): bfloat16
Number of layers: 28


In [16]:
evaluate_base = False

In [17]:
clear_cache()

In [19]:
# Evaluate the Base Model
if evaluate_base:
    clear_cache()
    
    # Start Timer
    start = time.time()
    
    BASE_MODEL_SCORES[base_model] = {}
    BASE_MODEL_SCORES[base_model]['batch_size'] = 1
    
    results = simple_evaluate(
                        model="hf",
                        model_args= {'pretrained': base_model,
                                     'trust_remote_code': True},
                        tasks = ['hellaswag', 'mutual', 'piqa', 'truthfulqa_mc2', 'gsm8k', 'lambada_openai', 'lambada_standard', 'winogrande', 'openbookqa'],
                        log_samples= True,
                        device= "cuda:0",
                        batch_size = 'auto:4',
                        limit = 100,
                        task_dict = ready_tasks,
                        apply_chat_template= False,
                        )

    # Stop Timer and save the result
    model_time = time.time() - start
    BASE_MODEL_SCORES[base_model]['time'] = model_time
    print("Base Model:", base_model, "\nTime Score:", model_time, "Seconds")

tokenizer_config.json:   0%|          | 0.00/3.06k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/28.1k [00:00<?, ?B/s]

model-00001-of-000002.safetensors:   0%|          | 0.00/8.61G [00:00<?, ?B/s]

model-00002-of-000002.safetensors:   0%|          | 0.00/6.62G [00:00<?, ?B/s]

...

bootstrapping for stddev: perplexity


100%|██████████| 100/100 [00:00<00:00, 153.21it/s]


bootstrapping for stddev: perplexity


100%|██████████| 100/100 [00:00<00:00, 153.57it/s]


Base Model: deepseek-ai/DeepSeek-R1-Distill-Qwen-7B 
Time Score: 1889.357815027237 Seconds


In [ ]:
# c_model = custom_model(base_model = base_model,
#                         dtype= dtype,
#                         num_layers= num_layers)
# clear_cache()
# run_ga(custom_model= c_model,
#        num_pruned_layers= 2,
#        tasks = ready_tasks,
#        metrics = metrics,
#        pop_size = 60,
#        max_gens = 10,
#        tourn_size= 5,
#        p_cx= 0.9,
#        p_mu= 0.5,
#        prev_results= {})